In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import os
import torch
import pandas as pd

import fastai
import dill
from fastai.imports import *
from fastai.vision import *
from fastai.vision.all import *
from fastai.metrics import error_rate, accuracy

In [ ]:
torch.cuda.get_device_name(0)

In [ ]:
fastai.__version__

## Folder Paths

In [ ]:
## Defining Directories
path = Path('../input/hotel-id-2021-fgvc8')

## csv
sample_sub = path/'sample_submission.csv'
labels = path/'train.csv'

## Reading csv submission

In [ ]:
train_data = pd.read_csv(labels).sample(frac=1, random_state = 2021).reset_index(drop = True)
train_data.head(5)

Creating Labels and changing images to include the new paths

In [ ]:
train_dir = './kaggle/working/train/'

train_data['image_path'] = train_dir + train_data.chain.astype('str') + '/' + train_data.image
train_data[:5]

## Unzip resized Images

In [ ]:
!tar xzvf /kaggle/input/hotelid-224/kaggle_hotelid_train_imgs_224x224.tgz

## Creating a model with the chain Id

In [ ]:
batch_size = 32
def create_dataLoader():
    dataset = ImageDataLoaders.from_df(df = train_data[['image_path', 'hotel_id']],
                                   path = '.',
                                   folder = '.',
                                   valid_pct=0.2,
                                   item_tfms=Resize(224, method='pad', pad_mode='reflection'),
                                   batch_tfms=aug_transforms(size=224),
                                   bs=batch_size)
    return dataset

In [ ]:
dataset = create_dataLoader()

In [ ]:
dataset.show_batch()

In [ ]:
learn = cnn_learner(dataset, resnet101, metrics=[accuracy, error_rate],   opt_func=QHAdam).to_fp16()

In [ ]:
%time

learn.fine_tune(12, 0.005, freeze_epochs=3)

In [ ]:
model_name ='model_Data_category_rn101'
learn.export(f'{model_name}.pkl', pickle_module=dill)

In [ ]:
# !rm -fr './kaggle/working/train'